In [1]:
print("hello, world!")

hello, world!


In [2]:
# system packages 
import os
import json

# internal packages 

# external packages
from openai import OpenAI
from dotenv import load_dotenv
from graphql import build_schema, parse
from jinja2 import Environment, FileSystemLoader

load_dotenv()
open_ai_api_key = os.getenv("OPENAI_API_KEY")

## GraphQL Schema Parsing

In [3]:

# with open('../assets/schemas/opensea/opensea_original_schema.graphql', 'r') as schema_file:
#     schema_str = schema_file.read()

# # TEMP: Try to parse the schema given the errors we are seeing
# custom_definitions = '''
# scalar BigDecimal
# scalar BigInt

# directive @entity on OBJECT
# directive @dailySnapshot on OBJECT
# directive @regularPolling on OBJECT
# directive @derivedFrom on OBJECT
# directive @transaction on OBJECT
# '''

# full_schema_str = custom_definitions + schema_str
# schema = build_schema(full_schema_str)

# for type_name, graphql_type in schema.type_map.items():
#     if type_name.startswith('__'):
#         continue  # Skip introspection types
#     print(f"Type: {type_name}")
#     for field_name, field in graphql_type.fields.items():
#         print(f"  Field: {field_name} (type: {field.type})")

## LangGraph

In [4]:
# import getpass
# import os

# if "OPENAI_API_KEY" not in os.environ:
#     os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
# open_ai_api_key = os.environ["OPENAI_API_KEY"]

In [5]:
# from typing import Annotated

# from langchain_openai import OpenAI
# from typing_extensions import TypedDict

# from langgraph.graph import StateGraph
# from langgraph.graph.message import add_messages


# class State(TypedDict):
#     messages: Annotated[list, add_messages]


# graph_builder = StateGraph(State)


# llm = OpenAI(model="gpt-4")


# def chatbot(state: State):
#     return {"messages": [llm.invoke(state["messages"])]}

# graph_builder.add_node("chatbot", chatbot)
# graph_builder.set_entry_point("chatbot")
# graph_builder.set_finish_point("chatbot")
# graph = graph_builder.compile()

In [6]:
# from IPython.display import Image, display

# try:
#     display(Image(graph.get_graph().draw_mermaid_png()))
# except Exception:
#     pass

## OpenAI Directly

In [7]:
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),  # This is the default and can be omitted
)

def gpt_chat_completion(client, message, model="gpt-4o"):
    return client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": message,
        }
    ],
    model=model,
)

print(gpt_chat_completion(client, "Hello! How are you?").choices[0].message.content)

Hello! I'm here and ready to help. How can I assist you today?


## Initial Test Set 

### Prompts 

- [ ] One entity quality vs. another entity quality 
- [ ] One column quality vs. another column quality 
- [ ] One schema quality vs. another schema quality 

### Demonstrations 

- [ ] A high quality entity (description)
- [ ] A high quality column (description)
- [ ] A high quality schema (description)

### Quality Categories

- [ ] 

In [8]:
with open("tests/assets/entity_comparison_assets.json", "r") as f:
    entity_comparison_assets = json.load(f)

gold_entity_comparison = "".join(entity_comparison_assets["gold"]["prompt"])
four_entity_comparison = "".join(entity_comparison_assets["four"]["prompt"])
three_entity_comparison = "".join(entity_comparison_assets["three"]["prompt"])
two_entity_comparison = "".join(entity_comparison_assets["two"]["prompt"])
one_entity_comparison = "".join(entity_comparison_assets["one"]["prompt"])

<Template 'entity_comparison_prompt.txt'>

In [44]:
# Function to reload the template
def reload_template(template_path="../assets/prompts", template_name="entity_comparison_prompt.txt"):
    env = Environment(loader=FileSystemLoader(template_path))
    env.cache.clear()
    entity_comparison_prompt_template = env.get_template(template_name)
    return entity_comparison_prompt_template

def parse_response(response): 
    response = response.choices[0].message.content
    response = response.strip('```json').strip('```').strip()
    response = json.loads(response)
    return response

example_prompt = reload_template()
example_prompt_filled = example_prompt.render({"entity_pred": "<example prediction goes here>", "entity_gold": "<example gold goes here>"})

entity_comparison_prompt_template = reload_template()
gold_output = entity_comparison_prompt_template.render({"entity_pred": {gold_entity_comparison}, "entity_gold": {gold_entity_comparison}})
four_output = entity_comparison_prompt_template.render({"entity_pred": {four_entity_comparison}, "entity_gold": {gold_entity_comparison}})
three_output = entity_comparison_prompt_template.render({"entity_pred": {three_entity_comparison}, "entity_gold": {gold_entity_comparison}})
two_output = entity_comparison_prompt_template.render({"entity_pred": {two_entity_comparison}, "entity_gold": {gold_entity_comparison}})
one_output = entity_comparison_prompt_template.render({"entity_pred": {one_entity_comparison}, "entity_gold": {gold_entity_comparison}})

print("Expecting output: 3 \n")
gold_comparison = parse_response(gpt_chat_completion(client, gold_output))
print(gold_comparison)

print("\n Expecting output: 4 \n")
four_comparison = parse_response(gpt_chat_completion(client, four_output))
print(four_comparison)

print("\n Expecting output: 3 \n")
three_comparison = parse_response(gpt_chat_completion(client, three_output))
print(three_comparison)

print("\n Expecting output: 2 \n")
two_comparison = parse_response(gpt_chat_completion(client, two_output))
print(two_comparison)

print("\n Expecting output: 1 \n")
one_comparison = parse_response(gpt_chat_completion(client, one_output))
print(one_comparison)  

Expecting output: 3 

{'reasoning': "First, we must compare the EVALUATION DOCUMENTATION with the GOLD DOCUMENTATION. Upon careful examination, both documentations are identical in their format and content. Each column in both documentations contains a full English sentence that clearly describes the meaning of the column, eliminating ambiguity. For example, all entries specify the context when necessary, such as 'The marketplace that this snapshot belongs to' rather than a less informative 'Market.' Also, they include concise yet comprehensive descriptions, such as explaining that a column represents 'Cumulative trade volume (in ETH).' The queries are syntactically correct, using proper capitalization and punctuation. Therefore, the provided documentation matches the GOLD documentation criteria and lacks any missing information, misleading descriptions, or grammar errors.", 'correctness': 3}

 Expecting output: 4 

{'reasoning': "The evaluation documentation provides detailed explanat

In [77]:
entity_comparison_prompt_template = reload_template(
    template_path="../assets/prompts",
    template_name="entity_comparison_revision.txt",
)

entity_comparison_prompt_modification = entity_comparison_prompt_template.render(
    {
        "original_prompt": {example_prompt_filled}, 

        "four_result_correct": {"True" if four_comparison["correctness"] == 4 else "False"},
        "four_result": {four_comparison["reasoning"]},
        
        "three_result_correct": {"True" if three_comparison["correctness"] == 3 else "False"},
        "three_result": {three_comparison["reasoning"]},
        
        "two_result_correct": {"True" if two_comparison["correctness"] == 2 else "False"},
        "two_result": {two_comparison["reasoning"]},
        
        "one_result_correct": {"True" if one_comparison["correctness"] == 1 else "False"},
        "one_result": {one_comparison["reasoning"]},
    }
)

prompt_revision = gpt_chat_completion(client, entity_comparison_prompt_modification)

In [87]:
file_path = "../assets/prompts/entity_comparison_prompt_revised.txt"

prompt_revision = parse_response(prompt_revision)
revised_prompt = prompt_revision["modified_prompt"]

# Replace the placeholder with Jinja syntax
revised_prompt = revised_prompt.replace(r"{entity_pred}", r"{{ entity_pred }}")
revised_prompt = revised_prompt.replace(r"{entity_gold}", r"{{ entity_gold }}")

with open(file_path, "w") as file:
    file.write(revised_prompt)